In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn  as sns
from tensorflow.keras import layers
import keras
from tensorflow.keras.layers import Rescaling, Resizing
from tensorflow.keras.layers import RandomRotation ,RandomCrop,RandomFlip
from tensorflow.keras.layers import Conv2D ,MaxPooling2D ,AveragePooling2D ,GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten , Dropout ,BatchNormalization ,LayerNormalization
from tensorflow.keras.optimizers import RMSprop ,Adagrad ,Adam
from tensorflow.keras.activations import sigmoid ,selu ,relu,softmax,tanh
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy


In [ ]:
IMAGE_SIZE=25
#loading the dataset
dataset= keras.utils.image_dataset_from_directory(
    "dataset" ,
    image_size=(IMAGE_SIZE ,IMAGE_SIZE),
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    bath_size=32,

)

In [ ]:
#checking the classes and the image_size and shape
class_names=dataset.n_classes
n_classes=3
print(dataset.shape)
for image_batch , label_batch in dataset.take(1):
  print(image_batch.numpy())
  print(label_batch.shape)

In [ ]:
#ploting the images
plt.figure(figsize=(10,5))
for image_batch , label_batch in dataset.take(1):
  for i in range(20):
    ax=plt.subplots(3,4,i+1)
    plt.imshow(image_batch[i].numpy().as_type("unint8"))
    plt.title(class_names[label_batch[i]])
    plt.axis('off')


In [ ]:
#resizing and rescaling the images
train_size=0.8
test_size=0.1
val_size=0.1

In [ ]:
train_size=len(dataset)*train_size
test_size=len(dataset)*test_size
val_size=len(dataset)*val_size

In [ ]:
train_ds=dataset.take(78)
len(train_ds)
test_ds=dataset.skip(78)
len(test_ds)
val_ds=dataset.skip(78)
len(test_ds)

In [7]:
#keep the images size and normalize the images
from tensorflow.keras import layers ,models ,Sequential
resize_and_reshape_img=Sequential([
        Resizing(256 ,256),
        Rescaling(1./255),
])

In [ ]:
#data agumentation
from tensorflow.keras.layers import RandomZoom ,RandomRotation ,RandomFlip

data_agumentation=Sequential([
    RandomCrop(2 ,2),
    RandomRotation(0.2),
    RandomFlip("horizontal_and_vertical"),
])

In [ ]:
import tensorflow as tf

train_ds = train_ds.map(
    lambda x, y: (data_agumentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
model=tf.keras.models.Sequential()

IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = n_classes
n_classes = 36
model=models.Sequential([
    layers.Conv2D(32 , kernel_size=(3,3) ,strides=(2,2) ,padding='same', activation='relu', kernel_initializer='he_normal' , input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS)),
    layers.MaxPooling2D(poolsize=(2,2) ,stride='valid'),
    layers.Conv2D(64 , kernel_size=(3,3) ,strides=(2,2) ,padding='same', activation='relu',kernel_initializer='he_normal'),
    layers.Dropout(0.2)
    layers.Conv2D(128 , kernel_size=(3,3) ,strides=(2,2) ,padding='same',activation='relu', kernel_initializer='he_normal'),
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(128 , kernel_size=(3,3) ,strides=(2,2) ,padding='same', activation='relu',kernel_initializer='he_normal'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Dropout((0.1))
    layers.Conv2D(128 , kernel_size=(3,3) ,strides=(2,2) ,padding='same',activation='relu', kernel_initializer='he_normal'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten()
    layers.Dense(64 , activation='relu'),
    layers.Dense(n_classes , activation='softmax'),
])

In [ ]:
callbacks=EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=0,
                        verbose=0,
                        mode='auto',
                        baseline=None,
                        restore_best_weights=True),

In [ ]:
model.compile(Adam(earning_rate=0.002, beta_1=0.8) ,SparseCategoricalCrossentropy() , metrics=['accuracy'])
history=model.fit(train_ds ,batch_size=32 , epochs=100 , callbacks=callbacks ,verbose=1  ,validation_data=val_ds)

In [ ]:
history.history["val_loss"]
history.history["loss"]
history.history["accuracy"]
history.history["val_accuracy"]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs[0,0]=plt.plot(history.history["val_loss"])
axs[0,0].set_title("val_loss_curve")
axs[0,0].set_xlabel("X_axis")
axs[0,0].set_ylabel("y_axis")

axs[0,1]=plt.plot(history.history["loss"])
axs[0,1].set_title("loss_curve")
axs[0,1].set_xlabel("X_axis")
axs[0,1].set_ylabel("y_axis")

axs[1,0]=plt.plot(history.history["accuracy"])
axs[1,0].set_title("loss_curve")
axs[1,0].set_xlabel("X_axis")
axs[1,0].set_ylabel("y_axis")

axs[1,1]=plt.plot(history.history["val_accuracy"])
axs[1,1].set_title("loss_curve")
axs[1,1].set_xlabel("X_axis")
axs[1,1].set_ylabel("y_axis")

In [ ]:
for label_batch ,image_batch  in dataset.take(1):
  first_image=image_batch[0].numpy().as_type("uint8")
  first_label=label_batch[0].numpy()
  print("prediction")
  plt.imshow(first_image)
  print(f"the actual_label:{first_label}")
  batch_prediction=model.predict(image_batch)
  print(f"predicted_label:{class_names[np.agrmax(batch_prediction[0])]}")

In [ ]:
def predict(model ,img):
  img_array=tf.keras.preprocessing.img_to_array(images[i].numpy()),
  img_array=tf.expand_dims(img_array ,0),
  prediction=model.predict(img_array)
  predict_class=class_names[np.argmax(prediction[0])]
  confidense=round(100 *(np.argmax(prediction[0]) ,2))
  return predict_class ,confidense



In [ ]:
plt.figure(figsize=(15,15))
for images ,labels in test_ds.take(1):
  for i in range(10):
    ax=plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('unit8'))
    predicted_class , confidense=predict(model , images[i].numpy())
    actual_class=class_names[labels[i]]
    plt.title(f"actual:{actual_class},\n Predicted:{predicted_class}.\n Confidense:{confidense}")
    plt.axis('off')


In [ ]:
model_version = 1
model.save(f"../models/{model_version}.keras")  # or use .h5 if you prefer
import os

model_version = 1
model_dir = "../models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model.save(f"{model_dir}/{model_version}.keras")